In [56]:
from exporter import ScholarExporter
import requests
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
from urllib.parse import quote
from datetime import date
import locale
from django.utils.encoding import smart_str#, smart_unicode

print(locale.getlocale())
if locale.getlocale()[0] is None:
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
    print(locale.getlocale())
## adapted from: https://github.com/TWRogers

('en_US', 'UTF-8')


In [99]:
main_url = "https://scholar.google.com/citations?user=tlMCbgwAAAAJ&sortby=pubdate"
re = requests.get(main_url)
#encoding = re.encoding if 'charset' in re.headers.get('content-type', '').lower() else None
parser = BeautifulSoup(re.content)#, "html.parser")                       
                       
scholar = parser.find('div', {'id': 'gsc_prf_in'}).encode("utf-8")#.text
papers = parser.body.find_all('tr', attrs={'class': 'gsc_a_tr'})

#parser.encode(

#escaper = BeautifulSoup.dammit.EntitySubstitution()
#escaped = escaper.substitute_html(parser)

In [107]:
PAPER_TEMPLATE = """
<div class="card">
  <div class="card-publication">
    <div class="card-body card-body-left">
      <p><a href="{url}">{title}</a></p>
      <p class="tab" style="font-style: italic;">by {authors}</p>
      <p class="tab"><b>{journal}</b></p>
    </div>
  </div>
  <div class="card-footer">
    <small class="tab">Published in <b>{year}</b>
  </small>
    </div>
</div>
"""

INTRO_TEXT = """
<div class="container">
  <div class="row clearfix layout layout-top-middle">
   <div class="col-xs-12 col-print-12">
    <a id="publications"></a> 
    <h3>Publications</h3>  
<p>Publications (<b>"""+ str(total)+"""</b>) last scraped on <b> """ + dateday + """</b> 
using <a href="https://github.com/TWRogers/google-scholar-export">google-scholar-export</a>.</p>
"""

In [108]:
parsed_papers = []
for paper in papers:
            #paper = paper.encode("utf-8")
            paper_soup = BeautifulSoup(str(paper))#, features="html.parser")
            print(paper_soup.find_all('div', {'class': 'gs_gray'})[0].text)
            try:
                citations_a = paper_soup.find('a', {'class': 'gsc_a_ac gs_ibl'})
                if citations_a is None:
                    citations_a = paper_soup.find('a', {'class': 'gsc_a_ac gs_ibl gsc_a_acm'})
                this_paper = {'title': paper_soup.find('a').text,
                              'year': paper_soup.find_all('span')[-1].text,
                              'n_citations': citations_a.text,
                              'citations_url': citations_a['href'],
                              'authors': paper_soup.find_all('div', {'class': 'gs_gray'})[0].text,
                              'journal': paper_soup.find_all('div', {'class': 'gs_gray'})[1].text,
                              'url': "https://scholar.google.com/" + paper_soup.find('a')['href'][1:]}

                if not this_paper['n_citations']:
                    print("no citations")
                    this_paper['n_citations'] = "0"
                
                if this_paper['journal'].endswith(', ' + this_paper['year']):
                    this_paper['journal'] = this_paper['journal'][:-len(', ' + this_paper['year'])]
                parsed_papers.append(this_paper)
            except KeyError:
                print('Warning: error parsing paper (key error).')
            except IndexError:
                print('Warning: error parsing paper.')
            except AttributeError:
                print('Warning: error parsing paper.')

R Gau, S Noble, K Heuer, KL Bottenhorn, IP Bilgin, YF Yang, ...
X Kobeleva, G Varoquaux, A Dagher, M Adhikari, C Grefkes, M Gilson
no citations
E Levitis, CG van Praag, R Gau, S Heunis, E DuPre, G Kiar, K Bottenhorn, ...
no citations
X Kobeleva, J Machts, M Veit, S Vielhaber, S Petri, MA Schoenfeld
no citations
T Fu, X Kobeleva, P Bronzlik, P Nösel, M Dadak, H Lanfermann, S Petri, ...
no citations
L Sannemann, AK Schild, S Altenstein, C Bartels, F Brosseron, K Buerger, ...
S Wolfsgruber, L Kleineidam, J Guski, A Polcher, I Frommann, S Roeske, ...
RA Husain, M Grimmel, M Wagner, JC Hennings, C Marx, RG Feichtinger, ...
X Kobeleva, A López-González, M Kringelbach, G Deco
no citations
MAF Fard, AP Rebelo, E Buglo, H Nemati, H Dastsooz, I Gehweiler, ...
AC Ludolph, J Schuster, J Dorst, L Dupuis, J Dreyhaupt, JH Weishaupt, ...
HJ Westeneng, TPA Debray, AE Visser, RPA van Eijk, JPK Rooney, ...
LR Peraza, R Cromarty, X Kobeleva, MJ Firbank, A Killen, S Graziadio, ...
X Kobeleva, M Firbank, L 

In [109]:
file = open("index.html","w")
parsed_papers_ascii = parsed_papers
total=len(parsed_papers)
dateday= date.today().isoformat()

file.write(INTRO_TEXT)
for paper in parsed_papers:
              file.write(PAPER_TEMPLATE.format(**paper))
file.close()

In [106]:
#s = ScholarExporter.from_user('tlMCbgwAAAAJ')  # Geoffrey Hinton user
#t = s.export('index.html')